## Bring up a GPU server

At the beginning of the lease time, we will bring up our GPU server. We will use the `python-chi` Python API to Chameleon to provision our server.

We will execute the cells in this notebook inside the Chameleon Jupyter environment.

In [1]:
from chi import server, context, lease
import os

context.version = "1.0"
context.choose_project()
context.choose_site(default="CHI@TACC")

Change the string in the following cell to reflect the name of *your* lease, then run it to get your lease:

In [3]:
l = lease.get_lease(f"gpu_p100_vn2233_2")
l.show()

HTML(value='\n        <h2>Lease Details</h2>\n        <table>\n            <tr><th>Name</th><td>gpu_p100_vn223…

Lease Details:
Name: gpu_p100_vn2233_2
ID: 3fdaa588-fba4-43b1-a8d8-9f596c64bb42
Status: ACTIVE
Start Date: 2025-02-27 17:20:00
End Date: 2025-02-28 17:19:00
User ID: fdb83aeea9e3eea047be316dd391717e977e5b381886f673cabf905e84e48a3b
Project ID: 477960c4206444c3a77b9e4ffa281ade

Node Reservations:
ID: 4db33479-c4ea-4f8e-8537-0fc3527471d8, Status: active, Min: 1, Max: 1

Floating IP Reservations:

Network Reservations:

Events:


The status should show as “ACTIVE” now that we are past the lease start time.

We will use this lease to bring up a server with the `CC-Ubuntu24.04-CUDA` disk image. This will take up to 10 minutes.

Note: security groups are not used at Chameleon bare metal sites, so we do not have to configure any security groups on this instance.

In [4]:
username = os.getenv('USER') # all exp resources will have this prefix
s = server.Server(
    f"multi_gpu-{username}-nyu", 
    reservation_id=l.node_reservations[0]["id"],
    image_name="CC-Ubuntu24.04-CUDA"
)
s.submit(idempotent=True)

Waiting for server multi_gpu-vn2233_nyu_edu-nyu's status to become ACTIVE. This typically takes 10 minutes, but can take up to 20 minutes.


Server has moved to status ACTIVE


Attribute,multi_gpu-vn2233_nyu_edu-nyu
Id,b178771d-3ac7-402c-a7da-c84cb16739b0
Status,ACTIVE
Image Name,CC-Ubuntu24.04-CUDA
Flavor Name,baremetal
Addresses,sharednet1: IP: 10.52.3.224 (v4) Type: fixed MAC: 44:a8:42:26:54:1f
Network Name,sharednet1
Created At,2025-02-27T17:20:57Z
Keypair,vn2233_nyu_edu-jupyter
Reservation Id,4db33479-c4ea-4f8e-8537-0fc3527471d8
Host Id,7c346a87f6e9c6f9d67fb076fe5d76102c3a39bab2eac416bf31a8b7


Then, we’ll associate a floating IP with the instance, so that we can access it over SSH.

In [5]:
s.associate_floating_ip()

In [6]:
s.refresh()
s.check_connectivity()

Checking connectivity to 129.114.108.65 port 22.


Connection successful


In [7]:
s.refresh()
s.show(type="widget")

Attribute,multi_gpu-vn2233_nyu_edu-nyu
Id,b178771d-3ac7-402c-a7da-c84cb16739b0
Status,ACTIVE
Image Name,CC-Ubuntu24.04-CUDA
Flavor Name,baremetal
Addresses,sharednet1: IP: 10.52.3.224 (v4) Type: fixed MAC: 44:a8:42:26:54:1f IP: 129.114.108.65 (v4) Type: floating MAC: 44:a8:42:26:54:1f
Network Name,sharednet1
Created At,2025-02-27T17:20:57Z
Keypair,vn2233_nyu_edu-jupyter
Reservation Id,4db33479-c4ea-4f8e-8537-0fc3527471d8
Host Id,7c346a87f6e9c6f9d67fb076fe5d76102c3a39bab2eac416bf31a8b7


## Set up Docker

To use common deep learning frameworks like Tensorflow or PyTorch, we can run containers that have all the prerequisite libraries necessary for these frameworks. Here, we will set up the container framework.

In [8]:
s.execute("curl -sSL https://get.docker.com/ | sudo sh")
s.execute("sudo groupadd -f docker; sudo usermod -aG docker $USER")
s.execute("docker run hello-world")

/opt/conda/lib/python3.10/site-packages/paramiko/client.py:889: UserWarning: Unknown ssh-ed25519 host key for 129.114.108.65: b'41e5025b3b391c9688c876ab2c4e4ac4'
  warnings.warn(


# Executing docker install script, commit: 4c94a56999e10efcf48c5b8e3f6afea464f9108e


+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install ca-certificates curl >/dev/null

Running kernel seems to be up-to-date.

The processor microcode seems to be up-to-date.

Restarting services...
 systemctl restart packagekit.service

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.
+ sh -c install -m 0755 -d /etc/apt/keyrings
+ sh -c curl -fsSL "https://download.docker.com/linux/ubuntu/gpg" -o /etc/apt/keyrings/docker.asc
+ sh -c chmod a+r /etc/apt/keyrings/docker.asc
+ sh -c echo "deb [arch=amd64 signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu noble stable" > /etc/apt/sources.list.d/docker.list
+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install docker-ce docker-ce-cli containerd.io docker-compose-plugin docker-ce-rootless-extras docker-buildx-pl

Client: Docker Engine - Community
 Version:           28.0.1
 API version:       1.48
 Go version:        go1.23.6
 Git commit:        068a01e
 Built:             Wed Feb 26 10:41:12 2025
 OS/Arch:           linux/amd64
 Context:           default

Server: Docker Engine - Community
 Engine:
  Version:          28.0.1
  API version:      1.48 (minimum version 1.24)
  Go version:       go1.23.6
  Git commit:       bbd0a17
  Built:            Wed Feb 26 10:41:12 2025
  OS/Arch:          linux/amd64
  Experimental:     false
 containerd:
  Version:          1.7.25
  GitCommit:        bcc810d6b9066471b0b6fa75f557a15a1cbf31bb
 runc:
  Version:          1.2.4
  GitCommit:        v1.2.4-0-g6c52b3f
 docker-init:
  Version:          0.19.0
  GitCommit:        de40ad0


To run Docker as a non-privileged user, consider setting up the
Docker daemon in rootless mode for your user:

    dockerd-rootless-setuptool.sh install

Visit https://docs.docker.com/go/rootless/ to learn about rootless mode.


T

Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world
e6590344b1a5: Pulling fs layer
e6590344b1a5: Verifying Checksum
e6590344b1a5: Download complete
e6590344b1a5: Pull complete
Digest: sha256:e0b569a5163a5e6be84e210a2587e7d447e08f87a0e90798363fa44a0464a1e8
Status: Downloaded newer image for hello-world:latest



Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/



<Result cmd='docker run hello-world' exited=0>

We will also install the NVIDIA container toolkit, with which we can access GPUs from inside our containers.

In [28]:
# get NVIDIA container toolkit 
s.execute("curl -fsSL https://nvidia.github.io/libnvidia-container/gpgkey | sudo gpg --dearmor -o /usr/share/keyrings/nvidia-container-toolkit-keyring.gpg \
  && curl -s -L https://nvidia.github.io/libnvidia-container/stable/deb/nvidia-container-toolkit.list | \
    sed 's#deb https://#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://#g' | \
    sudo tee /etc/apt/sources.list.d/nvidia-container-toolkit.list")
s.execute("sudo apt update")
s.execute("sudo apt-get install -y nvidia-container-toolkit")
s.execute("sudo nvidia-ctk runtime configure --runtime=docker")
s.execute("sudo systemctl restart docker")

deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://nvidia.github.io/libnvidia-container/stable/deb/$(ARCH) /
#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://nvidia.github.io/libnvidia-container/experimental/deb/$(ARCH) /


Get:1 https://nvidia.github.io/libnvidia-container/stable/deb/amd64  InRelease [1477 B]
Hit:2 http://security.ubuntu.com/ubuntu noble-security InRelease
Hit:3 https://download.docker.com/linux/ubuntu noble InRelease
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease [256 kB]
Get:5 https://nvidia.github.io/libnvidia-container/stable/deb/amd64  Packages [16.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2404/x86_64  InRelease
Get:7 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease [126 kB]
Hit:8 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports InRelease
Fetched 400 kB in 1s (360 kB/s)
Reading package lists...
Building dependency tree...
Reading state information...
241 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libnvidia-container-tools libnvidia-container

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 5811 kB in 0s (19.2 MB/s)
Selecting previously unselected package libnvidia-container1:amd64.
(Reading database ... 110603 files and directories currently installed.)
Preparing to unpack .../libnvidia-container1_1.17.4-1_amd64.deb ...
Unpacking libnvidia-container1:amd64 (1.17.4-1) ...
Selecting previously unselected package libnvidia-container-tools.
Preparing to unpack .../libnvidia-container-tools_1.17.4-1_amd64.deb ...
Unpacking libnvidia-container-tools (1.17.4-1) ...
Selecting previously unselected package nvidia-container-toolkit-base.
Preparing to unpack .../nvidia-container-toolkit-base_1.17.4-1_amd64.deb ...
Unpacking nvidia-container-toolkit-base (1.17.4-1) ...
Selecting previously unselected package nvidia-container-toolkit.
Preparing to unpack .../nvidia-container-toolkit_1.17.4-1_amd64.deb ...
Unpacking nvidia-container-toolkit (1.17.4-1) ...
Setting up nvidia-container-toolkit-base (1.17.4-1) ...
Setting up libnvidia-container1:amd64 (1.17.4-1) ...
Setting up lib

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

Running kernel seems to be up-to-date.

The processor microcode seems to be up-to-date.

No services need to be restarted.

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.
time="2025-02-26T23:06:11Z" level=info msg="Config file does not exist; using empty config"
time="2025-02-26T23:06:11Z" level=info msg="Wrote updated config to /etc/docker/daemon.json"
time="2025-02-26T23:06:11Z" level=info msg="It is recommended that docker daemon be restarted."


<Result cmd='sudo systemctl restart docker' exited=0>

## Cuda Execution Provider

In [9]:
s.execute("wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2404/x86_64/cuda-keyring_1.1-1_all.deb")

--2025-02-27 17:43:01--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2404/x86_64/cuda-keyring_1.1-1_all.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 23.221.22.214, 23.221.22.203, 23.221.22.217
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|23.221.22.214|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4328 (4.2K) [application/x-deb]
Saving to: ‘cuda-keyring_1.1-1_all.deb’

     0K ....                                                  100%  186M=0s

2025-02-27 17:43:01 (186 MB/s) - ‘cuda-keyring_1.1-1_all.deb’ saved [4328/4328]



<Result cmd='wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2404/x86_64/cuda-keyring_1.1-1_all.deb' exited=0>

In [10]:
s.execute("sudo dpkg -i cuda-keyring_1.1-1_all.deb && sudo apt-get update && sudo apt-get -y install cudnn")

(Reading database ... 110603 files and directories currently installed.)
Preparing to unpack cuda-keyring_1.1-1_all.deb ...
Unpacking cuda-keyring (1.1-1) over (1.1-1) ...
Setting up cuda-keyring (1.1-1) ...
Hit:1 https://download.docker.com/linux/ubuntu noble InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2404/x86_64  InRelease
Get:3 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease [256 kB]
Hit:4 http://security.ubuntu.com/ubuntu noble-security InRelease
Get:5 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease [126 kB]
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports InRelease [126 kB]
Get:7 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates/main amd64 Packages [883 kB]
Get:8 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates/main amd64 Components [151 kB]
Get:9 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates/universe amd64 Packages [1026 kB]
Get:10 http://nova.clouds.archive.ubu

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 987 MB in 37s (26.8 MB/s)
Selecting previously unselected package libcudnn9-cuda-12.
(Reading database ... 110603 files and directories currently installed.)
Preparing to unpack .../0-libcudnn9-cuda-12_9.7.1.26-1_amd64.deb ...
Unpacking libcudnn9-cuda-12 (9.7.1.26-1) ...
Selecting previously unselected package libcudnn9-dev-cuda-12.
Preparing to unpack .../1-libcudnn9-dev-cuda-12_9.7.1.26-1_amd64.deb ...
Unpacking libcudnn9-dev-cuda-12 (9.7.1.26-1) ...
Selecting previously unselected package libcudnn9-static-cuda-12.
Preparing to unpack .../2-libcudnn9-static-cuda-12_9.7.1.26-1_amd64.deb ...
Unpacking libcudnn9-static-cuda-12 (9.7.1.26-1) ...
Selecting previously unselected package cudnn9-cuda-12-8.
Preparing to unpack .../3-cudnn9-cuda-12-8_9.7.1.26-1_amd64.deb ...
Unpacking cudnn9-cuda-12-8 (9.7.1.26-1) ...
Selecting previously unselected package cudnn9-cuda-12.
Preparing to unpack .../4-cudnn9-cuda-12_9.7.1.26-1_amd64.deb ...
Unpacking cudnn9-cuda-12 (9.7.1.26-1) ...
Selecti

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

Running kernel seems to be up-to-date.

The processor microcode seems to be up-to-date.

No services need to be restarted.

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.


<Result cmd='sudo dpkg -i cuda-keyring_1.1-1_all.deb && sudo apt-get update && sudo apt-get -y install cudnn' exited=0>

## TenorRT Execution provider

In [11]:
s.execute("sudo apt-get -y install cuda-toolkit-12-8 tensorrt")

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  cuda-cccl-12-8 cuda-command-line-tools-12-8 cuda-compiler-12-8 cuda-crt-12-8
  cuda-cudart-12-8 cuda-cudart-dev-12-8 cuda-cuobjdump-12-8 cuda-cupti-12-8
  cuda-cupti-dev-12-8 cuda-cuxxfilt-12-8 cuda-documentation-12-8
  cuda-driver-dev-12-8 cuda-gdb-12-8 cuda-libraries-12-8
  cuda-libraries-dev-12-8 cuda-nsight-12-8 cuda-nsight-compute-12-8
  cuda-nsight-systems-12-8 cuda-nvcc-12-8 cuda-nvdisasm-12-8
  cuda-nvml-dev-12-8 cuda-nvprof-12-8 cuda-nvprune-12-8 cuda-nvrtc-12-8
  cuda-nvrtc-dev-12-8 cuda-nvtx-12-8 cuda-nvvm-12-8 cuda-nvvp-12-8
  cuda-opencl-12-8 cuda-opencl-dev-12-8 cuda-profiler-api-12-8
  cuda-sanitizer-12-8 cuda-toolkit-12-8-config-common
  cuda-toolkit-12-config-common cuda-toolkit-config-common cuda-tools-12-8
  cuda-visual-tools-12-8 gds-tools-12-8 libcublas-12-8 libcublas-dev-12-8
  libcufft-12-8 libcufft-dev-12-8 libcufile-12-8 libcu

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 8094 MB in 5min 53s (23.0 MB/s)
Selecting previously unselected package cuda-cccl-12-8.
(Reading database ... 110733 files and directories currently installed.)
Preparing to unpack .../00-cuda-cccl-12-8_12.8.55-1_amd64.deb ...
Unpacking cuda-cccl-12-8 (12.8.55-1) ...
Selecting previously unselected package cuda-cupti-12-8.
Preparing to unpack .../01-cuda-cupti-12-8_12.8.57-1_amd64.deb ...
Unpacking cuda-cupti-12-8 (12.8.57-1) ...
Selecting previously unselected package cuda-cupti-dev-12-8.
Preparing to unpack .../02-cuda-cupti-dev-12-8_12.8.57-1_amd64.deb ...
Unpacking cuda-cupti-dev-12-8 (12.8.57-1) ...
Selecting previously unselected package cuda-nvdisasm-12-8.
Preparing to unpack .../03-cuda-nvdisasm-12-8_12.8.55-1_amd64.deb ...
Unpacking cuda-nvdisasm-12-8 (12.8.55-1) ...
Selecting previously unselected package cuda-cuobjdump-12-8.
Preparing to unpack .../04-cuda-cuobjdump-12-8_12.8.55-1_amd64.deb ...
Unpacking cuda-cuobjdump-12-8 (12.8.55-1) ...
Selecting previously unsele

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

Running kernel seems to be up-to-date.

The processor microcode seems to be up-to-date.

No services need to be restarted.

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.


<Result cmd='sudo apt-get -y install cuda-toolkit-12-8 tensorrt' exited=0>

In the following cell, we will verify that we can see our NVIDIA GPUs from inside a container, by passing `--gpus-all`. (The `-rm` flag says to clean up the container and remove its filesystem when it finishes running.)

In [12]:
s.execute("  ")

<Result cmd='  ' exited=0>

## Triton server image

In [13]:
s.execute("docker pull nvcr.io/nvidia/tritonserver:24.10-py3")

24.10-py3: Pulling from nvidia/tritonserver
6414378b6477: Pulling fs layer
95632a7bc3a8: Pulling fs layer
4f4fb700ef54: Pulling fs layer
42fd5783a0f9: Pulling fs layer
86abba0172c5: Pulling fs layer
4af3f9032791: Pulling fs layer
e6f5e18001c2: Pulling fs layer
6da051397311: Pulling fs layer
c2ad6da399ba: Pulling fs layer
5738d44ce3f2: Pulling fs layer
42fd5783a0f9: Waiting
86abba0172c5: Waiting
ca8c93fac024: Pulling fs layer
b5e5a67ac549: Pulling fs layer
4af3f9032791: Waiting
9ded5c341569: Pulling fs layer
077e4aef015d: Pulling fs layer
1e885c3dbc29: Pulling fs layer
9189980ccfd7: Pulling fs layer
9ded5c341569: Waiting
580e3d1f6652: Pulling fs layer
077e4aef015d: Waiting
1e885c3dbc29: Waiting
5738d44ce3f2: Waiting
e6f5e18001c2: Waiting
76afc0ed6d6b: Pulling fs layer
c2ad6da399ba: Waiting
b5e5a67ac549: Waiting
88a8e4229993: Pulling fs layer
61e60250d71b: Pulling fs layer
6da051397311: Waiting
ee4d1a2201f9: Pulling fs layer
61e60250d71b: Waiting
7284127b8cc7: Pulling fs layer
e7a7c32481

<Result cmd='docker pull nvcr.io/nvidia/tritonserver:24.10-py3' exited=0>

## ====================== EOF for week 6 ===========================

## ==============================================

Let’s pull the actual container image that we are going to use, which is a Jupyter notebook server with PyTorch and CUDA libraries:

In [ ]:
# s.execute("docker rm -f torchnb; docker run -d -p 8888:8888 --gpus all --name torchnb quay.io/jupyter/pytorch-notebook:cuda12-pytorch-2.5.1")

There’s one more thing we must do before we can start out Jupyter server. Rather than expose the Jupyter server to the Internet, we are going to set up an SSH tunnel from our local terminal to our server, and access the service through that tunnel.

Here’s how it works: In your *local* terminal, run

    ssh -L 8888:127.0.0.1:8888 -i ~/.ssh/id_rsa_chameleon cc@A.B.C.D

where,

-   instead of `~/.ssh/id_rsa_chameleon`, substitute the path to your key
-   and instead of `A.B.C.D`, substitute the floating IP associated with your server

This will configure the SSH session so that when you connect to port 8888 locally, it will be forwarded over the SSH tunnel to port 8888 on the host at the other end of the SSH connection.

SSH tunneling is a convenient way to access services on a remote machine when you don’t necessarily want to expose those services to the Internet (for example: if they are not secured from unauthorized access).

Finally, run

In [ ]:
s.execute("docker logs torchnb")

Look for the line of output in the form:

    http://127.0.0.1:8888/lab?token=9c935d49c29ea28ad94a4b0b046ed93cffe3839a9ed76e0d

and copy it in your own browser to open the Jupyter environment.

In [ ]:
s.execute("docker logs torchnb")